In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.regression import *


def one_hot_encoder(df, nan_as_category=True):
    """質的変数をダミー変数に変換"""
    original_columns = list(df.columns)
    # dtypeがobject(i.e.str)の系列のカラム名をリストアップ
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    # pd.get_dummiesでダミー変数に変換
    # 変換するカラムにはcategorical_columnsを指定
    # pd.get_dummiesではpandas.DataFrameを指定すると[元の列名_カテゴリー名]に変換される
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def display_categorical_features(df, thresh=10):
    """データを最初にここに通して質的変数の確認を行う"""
    unique_count = df.select_dtypes('object').apply(pd.Series.nunique, axis=0)
    for idx in unique_count.index:
        if unique_count[idx] <= thresh:
            print(f"{idx}\t{unique_count[idx]}\t{df[idx].unique()}")
        else:
            print(f"{idx}\t{unique_count[idx]}")

In [3]:
df_train = pd.read_csv("train.csv").drop(["dateid"], axis=1)
display_categorical_features(df_train)

week	5	['Tue' 'Wed' 'Thu' 'Fri' 'Mon']
name	213
remarks	10	[nan 'お楽しみメニュー' '料理長のこだわりメニュー' 'スペシャルメニュー' '近隣に飲食店複合ビルオープン' '料理長イチオシ！'
 '料理長イチオシ!' '今月のイチオシ!' '今月のこだわりメニュー' 'コンビニ改装' '弁当種類増↓']
event	2	[nan 'ママの会' 'キャリアアップ支援セミナー']
weather	62


In [5]:
df_test = pd.read_csv("test.csv").drop(["dateid"], axis=1)
display_categorical_features(df_test)

week	5	['Mon' 'Tue' 'Wed' 'Thu' 'Fri']
name	106
remarks	2	[nan 'お楽しみメニュー' '人気のおすすめメニュー']
event	2	[nan 'ママの会' 'キャリアアップ支援セミナー']
weather	45


In [23]:
for df in [df_train, df_test]:
    weather_main = []
    weather_sub = []
    for w in df["weather"]:
        if "時々" in w:
            w_split = w.split("時々")
        elif "一時" in w:
            w_split = w.split("一時")
        elif "後" in w:
            w_split = w.split("後")
        else:
            w_split = [w]
        # if len(w_split[0]) >= 3:
        #     print(w)
        # if len(w_split) > 1 and len(w_split[1]) >= 3:
        #     print(w)
        if w_split[0] not in weather_main:
            weather_main.append(w_split[0])
        if len(w_split) > 1 and w_split[1] not in weather_sub:
            weather_sub.append(w_split[1])
    print(weather_main)
    print(weather_sub)
    print(f"main: {len(weather_main)}, sub: {len(weather_sub)}")

['晴', '薄曇', '大雨', '晴後', '快晴', '雨', '曇一時晴後', '曇', '雨後曇', '曇後', '雨後', '晴後曇', '雨、みぞれを伴う', '大雨後']
['薄曇', '晴', '曇一時雨、雷を伴う', '曇', '雨', '雨後晴', '雨、雷・ひょうを伴う', '雨後一時晴', '雨、雷を伴う', '大雨、雷を伴う', '雪後雨', '曇一時雨', '曇一時雪', 'みぞれ', '晴一時雨', '晴後一時雨、みぞれを伴う', '曇後一時雨', '曇、雷を伴う']
main: 14, sub: 18
['雨', '大雨', '大雨、雷を伴う', '晴', '曇後', '薄曇', '雨後', '曇', '快晴', '曇一時晴後', '晴後', '大雨後晴', '雨一時みぞれ後', '大雨後', '晴後雨', '薄曇後']
['曇', '晴', '雨', '曇、雷を伴う', '薄曇', '晴、みぞれを伴う', '雨一時晴', '雨後']
main: 16, sub: 8


In [27]:
dst_main = []
dst_sub = []
for df in [df_train, df_test]:
    weather_main = []
    weather_sub = []
    for w in set(df["weather"]):
        w_split = re.split("時々|一時|後|、|を伴う|・", w)
        main = w_split.pop(0)
        if main not in weather_main:
            weather_main.append(main)
        while len(w_split) > 0:
            sub = w_split.pop(0)
            if len(sub) == 0:
                continue
            if sub not in weather_sub:
                weather_sub.append(sub)
    print(weather_main)
    print(weather_sub)
    print(f"main: {len(weather_main)}, sub: {len(weather_sub)}")
    dst_main += weather_main
    dst_sub += weather_sub

dst_main = list(set(dst_main))
dst_sub = list(set(dst_sub))
print(dst_main)
print(dst_sub)
print(f"main: {len(dst_main)}, sub: {len(dst_sub)}")

['曇', '大雨', '雨', '薄曇', '晴', '快晴']
['晴', '雨', '曇', '雷', 'みぞれ', '雪', '薄曇', '大雨', 'ひょう']
main: 6, sub: 9
['曇', '大雨', '雨', '薄曇', '晴', '快晴']
['晴', '雨', '曇', 'みぞれ', '薄曇', '雷']
main: 6, sub: 6
['曇', '大雨', '晴', '雨', '快晴', '薄曇']
['曇', 'ひょう', '雪', '大雨', '雷', '晴', '雨', 'みぞれ', '薄曇']
main: 6, sub: 9


## 天気について
- 上記の通り、メインの天気(x6)と、サブの天気(x9)に分割、合計15特徴量に圧縮する

## remarksについて
- testには'お楽しみメニュー' '人気のおすすめメニュー'しかない
- trainに多く含まれていてもあまり意味をなさなそうなので以下の通り圧縮
  - '料理長のこだわりメニュー' '今月のこだわりメニュー' ⇒ 'こだわりメニュー'
  - '料理長イチオシ！' '料理長イチオシ!' '今月のイチオシ!' ⇒ 'イチオシ'
  - 'スペシャルメニュー'はtrainにしかなく、ランチビュッフェとメニューが特定できないため、この行は削除

In [28]:
a = set(df_train["name"]) & set(df_test["name"])
print(a)
for i in a:
    print(i)
print(f"common: {len(a)}, train_only: {len(set(df_train['name']) - a)}, test_only: {len(set(df_test['name']) - a)}")

{'酢豚', 'エビフライ', '炊き込みご飯・メンチカツ', '肉野菜オイスター炒め', '手作りチキンカツ', 'エビフライ・タルタル', '煮込みハンバーグ', 'サーモンフライ・自家製タルタル', '鶏肉の唐揚げ', '麻婆豆腐', 'タンドリーチキン', 'カレーライス', '豚肉の生姜焼き', '塩唐揚げ', 'チーズハンバーグ', '焼き肉ビビンバ', 'チキン香草焼き・マスタード', 'ポークカレー', 'チキン南蛮', '鶏肉黒胡椒焼きおろしソース', '回鍋肉', '油淋鶏', 'メンチカツ', '肉じゃが', '野菜ごろごろシチュー', '親子煮', 'チキンのレモンペッパー焼き', 'ポークスタミナ焼き', 'グリルチキンピクルス風味', 'カレーライス(ポークカレー)', '自家製手作りとんかつ', 'キーマカレー', 'チンジャオロース', 'チキン山賊やき', 'ポーク生姜焼き', 'カレーライス(チキンカレー)', 'ハンバーグ', '牛丼風', 'チキンカレー'}
酢豚
エビフライ
炊き込みご飯・メンチカツ
肉野菜オイスター炒め
手作りチキンカツ
エビフライ・タルタル
煮込みハンバーグ
サーモンフライ・自家製タルタル
鶏肉の唐揚げ
麻婆豆腐
タンドリーチキン
カレーライス
豚肉の生姜焼き
塩唐揚げ
チーズハンバーグ
焼き肉ビビンバ
チキン香草焼き・マスタード
ポークカレー
チキン南蛮
鶏肉黒胡椒焼きおろしソース
回鍋肉
油淋鶏
メンチカツ
肉じゃが
野菜ごろごろシチュー
親子煮
チキンのレモンペッパー焼き
ポークスタミナ焼き
グリルチキンピクルス風味
カレーライス(ポークカレー)
自家製手作りとんかつ
キーマカレー
チンジャオロース
チキン山賊やき
ポーク生姜焼き
カレーライス(チキンカレー)
ハンバーグ
牛丼風
チキンカレー
common: 39, train_only: 174, test_only: 67


In [29]:
b = set(df_train["name"]) - set(df_test["name"])
print(b)
print(len(b))
for i in b:
    print(i)

{'エビフライ・カキフライ', 'ポーク黒胡椒焼き', 'チキンカツ・マスタード', 'チキンのコーンクリーム焼き', 'ポークソテーデミきのこソース', 'チキンフリカッセ', 'ハンバーグ（デミきのこバター）', 'スキヤキ煮込み', '魚のマスタード焼き', '和風ハンバーグ', '牛肉コロッケ', '白身魚味噌焼き', 'ベルギー風チキンのクリーム煮', '豚の冷しゃぶ', 'カレイ唐揚げ夏野菜あん', 'かに玉・甘酢たれ', '豚肉の胡麻シャブ', '名古屋味噌カツ', '厚切イカフライ', 'マーボー豆腐', '焼き肉', '手作りチキンカツ特製ソース', '手作りひれかつ', 'たら竜田揚げ', '豚肉のマスタード焼き', 'チャプチェ', 'マス塩焼き', 'チキンステーキ・ミートソース', 'バーベキューチキン', '牛肉筍煮', 'ビーフストロガノフ', '豚肉の中華炒め', '鶏肉の照焼', '鶏のから揚げスイートチリソース', '手作りトンカツ', '鶏の味噌漬け焼き', '牛カルビ焼き肉', '自家製手作りチキンカツ', 'なすと挽肉のはさみ揚げ', 'えびフライ', 'ハンバーグ・マヨ照りソース', 'アジ唐揚げ南蛮ソース', '鶏肉の山賊焼き', 'チャンチャン焼き', '鯖塩焼き', '厚揚げ肉味噌炒め', 'ランチビュッフェ', '豚味噌メンチカツ', 'エビフライ・エビカツ', '鶏肉の治部煮風', '中華丼', 'ヒレカツ', 'チキンの辛味噌焼き', '和牛メンチカツ', '和風ソースハンバーグ', 'エビカツ', '厚切ハムカツ', '夏野菜カレー', '豚肉と白菜の中華炒め', 'チーズメンチカツ', '梅しそチキンロール', '鶏のカッシュナッツ炒め', 'ビーフトマト煮', '豚肉と茄子のピリ辛炒め', '鶏の照り焼きマヨ', '白身魚のマスタード焼き', 'ハンバーグ・ミートソース', 'ささ身チーズフライ', '肉団子のシチュー', '豚肉と玉子の炒め', 'カツ煮', 'カニクリームコロッケ', '白身魚のバジルソース', 'チキン山賊焼き', '汁なし担担麺', '白身魚の南部焼き', 'マーボ茄子', 'ポークソテー・オニオンソース', '鶏肉と野菜の黒胡椒炒め', 'サーモンフライ・タルタル', 'きのこデ

In [39]:
all_menu = set(df_train["name"]) | set(df_test["name"])
print(len(all_menu))
fish = []
seafood = []
chicken = []
pork = []
beef = []
vegetable = []
rice = []
noodle = []
another = []
for m in all_menu:
    if any(x in m for x in ("ご飯")):
        rice.append(m)
    elif any(x in m for x in ("麺")):
        noodle.append(m)
    if any(x in m for x in ("魚", "ぶり", "鯖", "カレイ", "鮭")):
        fish.append(m)
    elif any(x in m for x in ("エビ", "えび", "海老", "いか", "イカ")):
        seafood.append(m)
    elif any(x in m for x in ("鶏", "チキン", "鳥", "とり")):
        chicken.append(m)
    elif any(x in m for x in ("豚", "ぶた", "ハム", "ベーコン", "ポーク", "とんかつ")):
        pork.append(m)
    elif any(x in m for x in ("牛", "ビーフ", "すき焼き")):
        beef.append(m)
    elif "ハンバーグ" in m:
        pork.append(m)
        beef.append(m)
    # elif any(x in m for x in ("魚", "ぶり", "マス", "鯖", "カレイ")):
    #     vegetable.append(m)
    else:
        another.append(m)
print(f"fish: {len(fish)}, seafood: {len(seafood)}, chicken: {len(chicken)}, pork: {len(pork)}, beef: {len(beef)}, vegetable: {len(vegetable)}, rice: {len(rice)}, noodle: {len(noodle)}, another: {len(another)}")
print(fish)
print(seafood)
print(chicken)
print(pork)
print(beef)
print(vegetable)
print(rice)
print(noodle)
print(another)

280
fish: 30, seafood: 17, chicken: 63, pork: 62, beef: 39, vegetable: 0, rice: 13, noodle: 1, another: 88
['鮭の西京焼き', '魚のマスタード焼き', 'カレイ唐揚げ夏野菜あん', '赤魚味醂漬け焼き', 'カレイの唐揚げ', '白身魚フライ', 'カレイ唐揚げ野菜餡かけ', 'ぶりの照り焼き', '白身魚唐揚げ野菜餡かけ', '赤魚みりん漬け焼き', '白身魚のグリル・オリーブトマトソース', 'ぶり照焼き・根菜煮', 'カレイ唐揚げ野菜あんかけ', '白身魚ムニエル', 'カレイ唐揚げ', '焼魚野菜甘酢', '白身魚カレームニエル', '白身魚味噌焼き', 'ぶり照焼き', 'ぶり照り焼き・根菜添え', '鯖塩焼き', '白身魚のマスタード焼き', '白身魚のバジルソース', '白身魚の南部焼き', 'ぶり照り焼きと根菜煮', '魚のマスタード焼', 'ぶりレモンペッパー焼き', 'ぶり照焼', 'カレイ唐揚げ 甘酢あん', '五目炊き込みご飯・鮭の味噌漬け焼き']
['エビフライ・カキフライ', 'たっぷり海老カツ', '厚切イカフライ', 'エビフライ・タルタル', 'エビカツ・エビフライ', 'えびフライ×2本', 'エビグラタンカツ', '海老フライ・タルタルソース', 'エビフライ', '海老天ぷら', 'まるごとイカハンバーグ', '海老とイカのチリソース', 'えびフライ', 'エビフライ・エビカツ', 'エビカツ', 'エビ・イカチリソース', 'いか天ぷら']
['チキンカツ・マスタード', 'ベルギー風チキンのクリーム煮', '鶏肉のマスタード焼き', 'チキン香草焼き・マスタード', '手作りチキンカツ特製ソース', '油淋鶏', '鶏のから揚げスイートチリソース', 'チキン山賊やき', 'チキンカツ', 'チキンステーキおろしソース', '鶏肉の治部煮風', 'チキンの辛味噌焼き', '鶏のカッシュナッツ炒め', 'チキン山賊焼き', '鶏肉と野菜の黒胡椒炒め', '鶏のから揚げねぎ塩炒めソース', '鶏肉の唐揚げ', '鶏の天ぷら', '鶏の唐揚げおろしソース', 'チキン南蛮', 'チキンカレー', '鶏肉のカレー唐揚',

In [65]:
import jaconv

all_menu = list(set(df_train["name"]) | set(df_test["name"]))
for i in range(len(all_menu)):
    all_menu[i] = jaconv.kata2hira(all_menu[i])
print(len(all_menu))
fish = []
seafood = []
chicken = []
pork = []
beef = []
meat = []
vegetable = []
rice = []
noodle = []
another = []
for m in all_menu:
    if "ご飯" in m:
        rice.append(m)
    if any(x in m for x in ("麺", "春雨", "そば", "うどん")):
        noodle.append(m)
    if any(x in m for x in ("魚", "ぶり", "鯖", "かれい", "鮭", "さーもん", "かじき", "きす", "たら", "さば", "あじ", "ちゃんちゃん", "ほき")):
        fish.append(m)
    elif any(x in m for x in ("えび", "海老", "いか", "海鮮")):
        seafood.append(m)
    elif any(x in m for x in ("鶏", "ちきん", "鳥", "とり", "唐揚げ", "ぐりる", "親子")):
        chicken.append(m)
    elif any(x in m for x in ("豚", "ぶた", "はむ", "べーこん", "ぽーく", "とんかつ", "ひれかつ", "ささ身", "味噌かつ", "すたみな", "かつ煮")):
        pork.append(m)
    elif any(x in m for x in ("牛", "びーふ", "すき焼き", "すきやき", "めんちかつ", "ろーすかつ", "ぷるこぎ")):
        beef.append(m)
    elif any(x in m for x in ("はんばーぐ", "ろこもこ")):
        pork.append(m)
        beef.append(m)
    elif "ます" in m:
        fish.append(m)
    elif any(x in m for x in ("ころっけ", "ごーや", "菜", "とまと", "枝豆", "なす", "茄子", "筑前煮", "こーん")):
        vegetable.append(m)
    elif any(x in m for x in ("肉", "にく", "ちんじゃお")):
        meat.append(m)
    else:
        another.append(m)
print(f"fish: {len(fish)}, seafood: {len(seafood)}, chicken: {len(chicken)}, pork: {len(pork)}, beef: {len(beef)}, vegetable: {len(vegetable)}, rice: {len(rice)}, noodle: {len(noodle)}, another: {len(another)}")
print(fish)
print(seafood)
print(chicken)
print(pork)
print(beef)
print(meat)
print(vegetable)
print(rice)
print(noodle)
print(another)

280
fish: 42, seafood: 18, chicken: 68, pork: 71, beef: 48, vegetable: 20, rice: 11, noodle: 4, another: 19
['鮭の西京焼き', '魚のますたーど焼き', 'かれい唐揚げ夏野菜あん', 'あじ唐揚げ南蛮そーす', '赤魚味醂漬け焼き', 'かれいの唐揚げ', '厚切りほきふらい', '白身魚ふらい', 'さーもんふらい・自家製たるたる', 'かれい唐揚げ野菜餡かけ', 'ぶりの照り焼き', 'さーもんのむにえる2色そーす', 'きすと野菜の天ぷら', '白身魚唐揚げ野菜餡かけ', '赤魚みりん漬け焼き', '白身魚のぐりる・おりーぶとまとそーす', 'ぶり照焼き・根菜煮', 'かれい唐揚げ野菜あんかけ', '白身魚むにえる', 'かれい唐揚げ', '焼魚野菜甘酢', '白身魚かれーむにえる', 'さばの味醂干し焼き', '白身魚味噌焼き', 'ぶり照焼き', 'あじのみりん干し', 'たら竜田揚げ', 'ぶり照り焼き・根菜添え', 'ます塩焼き', 'ちゃんちゃん焼き', '鯖塩焼き', '白身魚のますたーど焼き', '白身魚のばじるそーす', '白身魚の南部焼き', 'さーもんふらい・たるたる', 'ぶり照り焼きと根菜煮', '魚のますたーど焼', 'ぶりれもんぺっぱー焼き', 'ぶり照焼', 'かじきの甘辛煮', 'かれい唐揚げ 甘酢あん', '五目炊き込みご飯・鮭の味噌漬け焼き']
['えびふらい・かきふらい', 'たっぷり海老かつ', '厚切いかふらい', '海鮮ちりそーす', 'えびふらい・たるたる', 'えびかつ・えびふらい', 'えびふらい×2本', 'えびぐらたんかつ', '海老ふらい・たるたるそーす', 'えびふらい', '海老天ぷら', 'まるごといかはんばーぐ', '海老といかのちりそーす', 'えびふらい', 'えびふらい・えびかつ', 'えびかつ', 'えび・いかちりそーす', 'いか天ぷら']
['ちきんかつ・ますたーど', 'べるぎー風ちきんのくりーむ煮', '塩唐揚げ', '鶏肉のますたーど焼き', 'ちきん香草焼き・ますたーど', '手作りちきんかつ特製そーす', '油淋鶏', '鶏のから揚げすいーとちりそーす', 'ちきん山

In [79]:
cook_yaki = []
cook_ni = []
cook_age = []
cook_itame = []
cook_curry = []
cook_another = []
for m in all_menu:
    if any(x in m for x in ("煮", "すき")):
        cook_ni.append(m)
    elif any(x in m for x in ("焼", "やき", "がーりっく", "ぐりる", "すてーき", "はんばーぐ")):
        cook_yaki.append(m)
    elif any(x in m for x in ("揚", "ふらい", "油", "あげ", "かつ", "ころっけ", "天ぷら")):
        cook_age.append(m)
    elif any(x in m for x in ("炒", "そてー")):
        cook_itame.append(m)
    elif "かれー" in m:
        cook_curry.append(m)
    else:
        cook_another.append(m)
print(f"yaki: {len(cook_yaki)}, ni: {len(cook_ni)}, age: {len(cook_age)}, itame: {len(cook_itame)}, curry: {len(cook_curry)}, another: {len(cook_another)}")
print(cook_yaki)
print(cook_ni)
print(cook_age)
print(cook_itame)
print(cook_curry)
print(cook_another)

yaki: 81, ni: 24, age: 87, itame: 22, curry: 9, another: 57
['鮭の西京焼き', '魚のますたーど焼き', '鶏肉のますたーど焼き', 'ちきん香草焼き・ますたーど', 'はんばーぐ・まよ照りそーす', 'ちきん山賊やき', 'ちきんすてーきおろしそーす', '赤魚味醂漬け焼き', 'ちきんの辛味噌焼き', '和風そーすはんばーぐ', 'はんばーぐ・みーとそーす', 'ちきん山賊焼き', 'きのこでみそーすはんばーぐ', 'すたみなまよはんばーぐ', 'ぶりの照り焼き', 'きのこそーすはんばーぐ', '赤魚みりん漬け焼き', '豚肉の香味焼き', '白身魚のぐりる・おりーぶとまとそーす', 'まーぼー焼きそば', '鶏肉の照り焼き', 'がーりっくとまとちきん', '豚ろーすのぴざ風ちーず焼き', '豚肉の生姜焼き', '鶏肉の胡麻味噌焼き', 'ちきんのがーりっくばたーそーす', 'おろしはんばーぐそーすと', '焼魚野菜甘酢', 'ちきんすてーき・和風そーす', '鶏肉黒胡椒焼き', 'まるごといかはんばーぐ', 'はんばーぐとまとそーす', '豚肉の味噌焼き', 'ちきん香草焼きますたーどそーす', 'ぽーく黒胡椒焼き', 'びびんば風焼き肉', 'さばの味醂干し焼き', 'ちきんのこーんくりーむ焼き', 'はんばーぐ（でみきのこばたー）', '和風はんばーぐ', '白身魚味噌焼き', 'ぶり照焼き', '焼き肉', '豚肉のますたーど焼き', 'ぶり照り焼き・根菜添え', 'ます塩焼き', 'ちきんすてーき・みーとそーす', '鶏肉の照焼', 'はんばーぐ・まよ照そーす', '鶏の味噌漬け焼き', 'おろしそーすはんばーぐ', '牛かるび焼き肉', '鶏肉の山賊焼き', 'ちゃんちゃん焼き', '鯖塩焼き', 'ぽーくすたみな焼', '鶏肉黒胡椒焼きおろしそーす', '鶏の照り焼きまよ', '白身魚のますたーど焼き', '白身魚の南部焼き', 'ちきんすてーき和風そーす', 'ぽーく生姜焼き', '鶏肉の塩麹焼き', '魚のますたーど焼', 'ちきんすてーき', 'ぶりれもんぺっぱー焼き', 'きみとろはんばーぐ', 'ちきんのれもんぺっぱー焼き', 'ぐりるちきんぴくるす風味', 'はんばーぐ'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("train.csv")
for name, group in df.groupby("soldout"):
    if name == 1:
        df_soldout = group
print(df_soldout.head())

       dateid      y week  soldout        name   kcal remarks event  payday  \
0  2014-04-01  125.0  Tue        1       メンチカツ  377.0     NaN   NaN     NaN   
2  2014-04-03  100.0  Thu        1          酢豚  390.0     NaN   NaN     NaN   
3  2014-04-04   90.0  Fri        1  和風ソースハンバーグ  350.0     NaN   NaN     NaN   
5  2014-04-08   96.0  Tue        1    肉団子のシチュー  353.0     NaN   NaN     NaN   
6  2014-04-09   88.0  Wed        1        青椒肉絲  376.0     NaN   NaN     NaN   

        weather  precipitation  temperature  
0         晴時々薄曇            0.0         13.9  
2            大雨           65.0         13.8  
3  晴時々曇一時雨、雷を伴う           40.0         15.3  
5            快晴            0.0         15.8  
6        晴後一時薄曇            0.0         15.5  


In [7]:
df_dst = df_soldout["y"]
df_dst.head()

0    125.0
2    100.0
3     90.0
5     96.0
6     88.0
Name: y, dtype: float64

In [8]:
df_dst.to_csv("train_soldout.csv")